<a href="https://colab.research.google.com/github/jishnujayakumar/CPD-2021/blob/main/Change_Point_Detection_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Helpful pointers**

1.   https://mathinsight.org/definition/node_degree
2.   cupy-cuda101==v9.0.0b1 - https://github.com/cupy/cupy/issues/4516

Observations
Numpy version takes 37.1 sec for 1 sampling process, CuPy version of the same takes 114 sec. Hence, going forward with numpy and numba.



In [32]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [33]:
!pip install numpy numba tqdm tensorflow-gpu #dask

     |████████████████████████████████| 394.3MB 44kB/s 


In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


In [2]:
!nvidia-smi

Fri Feb  5 15:48:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Import requried packages
import numpy as np

# import cupy as np
# import numpy as npy

from tqdm import tqdm_notebook as tqdm

import dask # to allow parallel computation

import scipy
from scipy.stats import norm
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree

from numba import jit, njit, prange, vectorize, float64

import warnings 
warnings.filterwarnings('ignore')
# warnings.resetwarnings() # Uncomment to turn on wanrings

import tensorflow as tf
tf.autograph.set_verbosity(0)

In [48]:
# TODO: create a new exception class for dist_not_found_error
def raise_dist_not_found_error(dist_name):
  raise Exception("Incorrect distribution name")

# Sample data points based on a particular distribution
def sample_data(dist_name, n_samples):
  # ADD more distributions as per need
  if dist_name == "normal": 
    return np.random.normal(500, 20, n_samples)
  else:
    raise_dist_not_found_error(dist_name)

def p1(t, n):
  return (2.0 * t * (n-t)) / (n * (n-1))

def p2(t, n):
  return (4.0 * t * (t-1) * (n-t) * (n-t-1)) / (n * (n-1) * (n-2) * (n-3))

# Expectation of RG(t)
def E_RG(t, adj_mat):
  n = get_num_vertices(adj_mat)
  return p1(t, n) * get_num_edges(adj_mat)

# Variance of RG(t)
def Var_RG(t, adj_mat):
  n = get_num_vertices(adj_mat)
  p1t = p1(t, n)
  p2t = p2(t, n)
  G = get_num_edges(adj_mat)
  first_term = p2t * G

  Gi = 0

  # TODO: Vectorize this by Gi=np.apply_along_axis(get_node_degree, 1, adj_mat) is very slow
  for node_index in prange(n):
    Gi += get_node_degree(adj_mat, node_index)

  # Gi=np.apply_along_axis(np.count_nonzero, 1, adj_mat) #very slow

  second_term = ((0.5* p1t) - p2t) * np.sum(Gi**2)
  third_term = ( p2t - (p1t**2) ) * (G**2)
  return first_term + second_term + third_term

# Number of edges in graph: G => adj_mat
def get_total_edges(adj_mat):
  return np.count_nonzero(adj_mat)/2 # for undirected graph

def get_num_edges(adj_mat):
  # For sub graph
  return np.count_nonzero(adj_mat)

# Degree of node i
def get_node_degree(adj_mat, node_index):
  return np.count_nonzero(adj_mat[node_index])

def get_num_vertices(adj_mat):
  return adj_mat.shape[0]

# R G (t)
# is the number of edges in the graph G that connect observations from the “past”
# (≤ t) to the “future” (> t).

def RG(t, adj_mat):
  # TODO: complete this from ref paper
  # Get the sub graph/adj_matrix consisting of [:t+1,t:]
  return get_num_edges(adj_mat[:t+1,t:])

# Calculate zscore
# TODO:vectorize this
def zscore(arr, mean, var):
  return ( (arr - mean) / var )



#TF
def bootstrap_tf(Dn, n):
  # print(f"\ri:{i}")


  # Dn = t2np_array(Dn)
  Dnew = resample(Dn, n)
  
  print(f"Dnew: {tf.shape(Dnew)}")

  # n=tf.shape(Dnew)
  # Calculate Scan Statisics for resampled Data
  return get_scan_statistics(Dnew , n, 2 , n)

def simulate(dist, num_simulations):
  Dn_sizes = []
  initial_samples = 150
  bootstrap_runs = 10**2
  for _ in tqdm(prange(num_simulations)):
    # Simulation
    X = sample_data(dist, initial_samples)
    Dn_size = cpd(X, bootstrap_runs)
    print(Dn_sizes, Dn_size)
    Dn_sizes.append(Dn_size)
  
  return  np.array(Dn_sizes)

@tf.function
def run_bootstrap_graph(Dn, n):
  result = tf.Variable(np.zeros([n], dtype=np.float32))
  i = tf.constant(0, dtype=tf.int32)
  while tf.less(i, 10):
    result[i].assign(bootstrap_tf(Dn, n))  # Performance may require tuning here.
    i += 1
  return result

# Change point detection mechanism
def cpd(D_N0, bootstrap_runs):
  Dn = D_N0
  N0 = D_N0.size
  while True:
    # Draw new observation Dx
    Dx = sample_data(dist, 1)[0]
    
    # Adding new observation to existing Data
    Dn = np.append(Dn , Dx) 
    
    #Number of observation in Dn
    n = Dn.size

    # print(f"\r {n}")

    # Get scan statistic
    Z = get_scan_statistics(Dn, n, N0 , n)


    # ================================ RUN GRAPH ====================================
    # result = tf.Variable(np.zeros([bootstrap_runs], dtype=np.float16))

    # @tf.function(experimental_relax_shapes=True)
    # def run_graph():
    #   i = tf.constant(0, dtype=tf.int32)
    #   while tf.less(i, bootstrap_runs):
    #     result[i].assign(bootstrap_tf(Dn))  # Performance may require tuning here.
    #     i += 1

    # run_graph()
    # result = result.read_value()
    # proto_tensor = tf.make_tensor_proto(result)  # convert `tensor result` to a proto tensor
    # result = tf.make_ndarray(proto_tensor) 
    # # print(result, result.size)
    # critZ = np.sort(result)[-1]
    # ===============================================================================

    result = run_bootstrap_graph(Dn, n)
    result = result.read_value()
    # proto_tensor = tf.make_tensor_proto(result)  # convert `tensor result` to a proto tensor
    # result = tf.make_ndarray(proto_tensor) 
    # print(result, result.size)
    critZ = np.sort(result)[-1]

    # Stopping rule
    if not (Z < critZ):
      # print(Z >= critZ)
      # print(f"Z:{Z}, \n critZ:{critZ}")
      break
    
  return Dn.size



# TF
def resample(D, n):
  # print(D_idx)
  # with tf.compat.v1.Session()  as sess:
  #   n=tf.shape(D).eval()[0]
  #   print(n)
  sampled_indices=tf.experimental.numpy.random.randint(low=0, high=n, size=n, dtype=tf.int32)

  # print(D)
  # # Converting Tensor to TensorProto 
  # proto = tf.make_tensor_proto(D)   

  # print(D)
  # # Generating numpy array 
  # D = tf.make_ndarray(proto) 
  # print(D)

  return tf.experimental.numpy.take(D, sampled_indices)

# TF tensor to numpy array
def t2np_array(tf_tensor):
  # Converting Tensor to TensorProto 
  proto = tf.make_tensor_proto(tf_tensor) 
  # Generating numpy array 
  return tf.make_ndarray(proto) 

# Get adjacency matrix
@tf.function
def get_adjacency_matrix(arr):
  # TODO: check this
  # Calculate pairwise distance
  a = tf.zeros([0, tf.shape(arr)[0]])
  ret_arr=[]

  for data_point in arr:
    ret_arr.append(np.abs(arr - data_point))  

  np.array(ret_arr)

  # flattened_unpacked = tf.unstack(tf.reshape(arr, [-1]))
  # for elem in flattened_unpacked:
  #     ret_arr.append(tf.abs(flattened_unpacked - elem)) # absolute distance
  # return tf.constant(ret_arr).numpy()

def call_py(func_name, input):
  return tf.py_function(func=func_name, inp=[input], Tout=tf.float32)

def get_MST(adj_mat):
  """
  Purpose: Generate Minimum Spanning Tree (MST)
  Input: Graph adjacency matrix
  Output: MST
  """

  mst = minimum_spanning_tree(adj_mat)
  # mst = minimum_spanning_tree(adj_mat.get())
  return mst.toarray()

# Calculate Scan Statistics value for given Dn
def get_scan_statistics(Dn, n, n0, n1):

  # Dn=t2np_array(Dn)

  # pdist_mat = tf.py_function(func=get_adjacency_matrix, inp=[Dn], Tout=tf.float32)
  pdist_mat = call_py(get_adjacency_matrix, Dn)

  #Make this faster by using numba on GPU
  mst = call_py(get_MST, x)

  # mst = np.asarray(get_MST(pdist_mat)) # Move to GPU from host

  # print(mst.get_shape())

  t=10

  # calculate zcores for RG
  RG_vec=RG(t, mst)
  mean=E_RG(t, mst)
  var=Var_RG(t, mst)
  zRG = zscore(RG_vec,mean,var)

  # Maximum of all Z values between n0 and n1
  return np.amax(zRG)

# Re sample data based on index (perform random func on index)
# def resample(D):
#   sampled_indices = np.random.randint(D.size, size=D.size)
#   return np.take(D, sampled_indices)

In [49]:
# Select seed value
seed_value=1
 
# Set seed for random number generation
np.random.seed(seed_value)
 
# Test on normal distribution for now
dist="normal"
 
# %timeit Dn_sizes = simulate(dist, 100)
 
Dn_sizes = []
initial_samples = 150
num_simulations = 1
bootstrap_runs = 10**2
 
for _ in tqdm(range(num_simulations)):
  # Simulation
  X = sample_data(dist, initial_samples)
  print(cpd(X, bootstrap_runs))
#   Dn_size = cpd(X, bootstrap_runs)
#   Dn_sizes.append(Dn_size)
 
# print(Dn_sizes, Dn_size)
# Dn_sizes = np.array(Dn_sizes)
# np.mean(Dn_sizes)
 
# ====================== To run simulations in parallel =====================================
# sim_results = tf.Variable(np.zeros([num_simulations], dtype=np.float16))
 
# @tf.function
# def run_simulation_graph():
#   i = tf.constant(0, dtype=tf.int32)
#   while tf.less(i, num_simulations):
#     X = sample_data(dist, initial_samples)
#     sim_results[i].assign(cpd(X))  # Performance may require tuning here.
#     i += 1
 
# sim_results=run_simulation_graph()
# sim_results = sim_results.read_value()
# proto_tensor = tf.make_tensor_proto(sim_results)  # convert `tensor result` to a proto tensor
# sim_results = tf.make_ndarray(proto_tensor) 
# np.mean(sim_results)

UnimplementedError: ignored

In [41]:
import tensorflow as tf
import numpy as np
input = [1, 9, 2, 1, 0]

a=tf.constant([input])
b=tf.transpose(a)

tf.matmul(b,a)

ret_arr=[]
flattened_unpacked = tf.unstack(tf.reshape(a, [-1]))
for elem in flattened_unpacked:
    ret_arr.append(np.abs(flattened_unpacked - elem)) # absolute distance
x=tf.constant(ret_arr).numpy()

# x = tf.compat.v1.placeholder(tf.float32)
# m = tf.compat.v1.placeholder(tf.float32)

y = tf.py_function(func=get_MST, inp=[x], Tout=tf.float32)
y

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[0., 0., 1., 0., 1.],
       [0., 0., 0., 0., 0.],
       [0., 7., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)>

In [ ]:
n=10
tf.experimental.numpy.random.randint(low=0, high=n, size=n, dtype=int)